In [2]:
import pandas as pd
from PIL import Image
import torch
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms,models

In [3]:
data_map=pd.read_csv(r'C:\Users\ihhim\OneDrive\Desktop\project1\data\chest_xray\metadata.csv')

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_map['encoded_class'] = le.fit_transform(data_map['class'])

In [5]:
class xraydataset(Dataset):
    def __init__(self,df_split,transform):
        self.df_split=df_split
        self.transform=transform
        
    def __len__(self):
        return len(self.df_split)  
    
    def __getitem__(self,index):  
        img_path=self.df_split.iloc[index]['image_path']
        label=self.df_split.iloc[index]['encoded_class']

        image=Image.open(img_path).convert('L')
        img_trans=self.transform(image)

        return img_trans,label

In [6]:
train_dataset=xraydataset(data_map[data_map['split']=='train'],transform=transforms.ToTensor())
val_dataset=xraydataset(data_map[data_map['split']=='val'],transform=transforms.ToTensor())
train_dataloader=DataLoader(train_dataset,batch_size=32,shuffle=True)
val_dataloader=DataLoader(val_dataset,batch_size=32,shuffle=False)

In [7]:
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, padding=1),   
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(16, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Flatten(),
    nn.Linear(64 * 16 * 16, 128),  
    nn.ReLU(),
    nn.Linear(128, 2)              
)

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


Sequential(
  (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten(start_dim=1, end_dim=-1)
  (10): Linear(in_features=16384, out_features=128, bias=True)
  (11): ReLU()
  (12): Linear(in_features=128, out_features=2, bias=True)
)

In [8]:
class_counts = data_map[data_map['split']=='train']['encoded_class'].value_counts().sort_index().values
weights = torch.tensor(1.0 / class_counts, dtype=torch.float).to(device)
loss = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [9]:
from tqdm import tqdm
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for images, labels in tqdm(train_dataloader, total=len(train_dataloader)):
        images, labels = images.to(device), labels.to(device).long()
        optimizer.zero_grad()
        preds = model(images)
        loss_val = loss(preds, labels)
        loss_val.backward()
        optimizer.step()
        running_loss += loss_val.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_dataloader):.4f}")

    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device).long()
            outputs = model(images)
            loss_batch = loss(outputs, labels)
            val_loss += loss_batch.item()
    val_loss /= len(val_dataloader)
    print(f"Validation Loss: {val_loss:.4f}")

100%|██████████| 243/243 [00:59<00:00,  4.12it/s]


Epoch 1/10, Loss: 0.4470
Validation Loss: 0.4221


100%|██████████| 243/243 [00:11<00:00, 20.59it/s]


Epoch 2/10, Loss: 0.1342
Validation Loss: 0.3368


100%|██████████| 243/243 [00:12<00:00, 19.85it/s]


...

Epoch 8/10, Loss: 0.0617
Validation Loss: 0.4609


100%|██████████| 243/243 [00:13<00:00, 18.24it/s]


Epoch 9/10, Loss: 0.0584
Validation Loss: 0.1814


100%|██████████| 243/243 [00:12<00:00, 19.20it/s]

Epoch 10/10, Loss: 0.0570
Validation Loss: 0.5299


In [10]:
from sklearn.metrics import classification_report
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in val_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
print(classification_report(all_labels, all_preds, target_names=le.classes_))

              precision    recall  f1-score   support

      NORMAL       1.00      0.62      0.77         8
   PNEUMONIA       0.73      1.00      0.84         8

    accuracy                           0.81        16
   macro avg       0.86      0.81      0.81        16
weighted avg       0.86      0.81      0.81        16



In [11]:
torch.save(model.state_dict(), r'C:\Users\ihhim\OneDrive\Desktop\project1\models\cnn_model.pth')